In [ ]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from os import path
import os

root_path = "/content/drive/MyDrive/DFS/pu/thesis/the_new_one"

survival_path = path.join(root_path, "survival")
dataset_path = path.join(survival_path, "data")
model_path = path.join(survival_path, "models")

os.makedirs(dataset_path, exist_ok=True)
os.makedirs(model_path, exist_ok=True)

!ls "$dataset_path"

comment.zip		       df_final_kmean.csv  df_numeric_augment.csv
df_aspect_analysis_expand.csv  df_imb.csv	   df_numeric.csv
df_final.csv		       df_numeric_aug.csv  pu.csv


# Conditional survival forest model

In [ ]:
%%capture

try:
    import pysurvival, survivors
except:
    !pip install git+https://github.com/it-dainb/pysurvival.git
    !pip install git+https://github.com/it-dainb/dev-survivors.git

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
from survivors import constants as cnt

df = pd.read_csv(path.join(dataset_path, "df_numeric_aug.csv"))
df = df.rename({"survival_duration_days": cnt.TIME_NAME, "closed": cnt.CENS_NAME}, axis=1)

features = list(df.columns)
features.remove(cnt.TIME_NAME)
features.remove(cnt.CENS_NAME)

categ = ['price']

y = cnt.get_y(cens=df[cnt.CENS_NAME], time=df[cnt.TIME_NAME])
X = df.loc[:, features]

if y[cnt.TIME_NAME].min() == 0:
    y[cnt.TIME_NAME] += 1

X.shape, y.shape

((315, 11), (315,))

In [ ]:
from pysurvival.models.survival_forest import ConditionalSurvivalForestModel as PySurvConditionalSurvivalForestModel
from pysurvival.models.multi_task import NeuralMultiTaskModel as PySurvNeuralMultiTaskModel
from pysurvival.models.semi_parametric import NonLinearCoxPHModel as PySurvNonLinearCoxPHModel

from sksurv.functions import StepFunction
import json, os

import numpy as np

def encode_params(params):
    encoded_params = {}
    for key, vals in params.items():
        for att, val in vals.items():
            encoded_params[f"{key}|{att}"] = val if isinstance(val, list) else [val]
    return encoded_params

def decode_params(params):
    decoded_params = {}
    for key, val in params.items():
        key, att = key.split("|")

        if key not in decoded_params:
            decoded_params[key] = {}

        decoded_params[key][att] = val

    return decoded_params

class PySurviavalWrapper(object):
    base_model = None

    def __init__(self, **kwargs):
        self.model = None

        kwargs = decode_params(kwargs)

        self.init_kwargs = kwargs.get("init", {})
        self.fit_kwargs = kwargs.get("fit", {})

        if "activation" in self.init_kwargs:
            self.init_kwargs["structure"] = [
                {
                    "activation": self.init_kwargs["activation"],
                    "num_units": self.init_kwargs["num_units"]
                }
                for _ in range(self.init_kwargs['layers'])
            ]

            self.init_kwargs.pop("activation")
            self.init_kwargs.pop("num_units")
            self.init_kwargs.pop("layers")

    def fit(self, X, y):

        T = y[cnt.TIME_NAME]
        E = y[cnt.CENS_NAME]

        self.model = self.base_model(**self.init_kwargs)
        self.model.fit(X, T, E, **self.fit_kwargs)

        return self

    def _array_to_step_function(self, array):
        n_samples = array.shape[0]
        funcs = np.empty(n_samples, dtype=np.object_)

        for i in range(n_samples):
            funcs[i] = StepFunction(x=np.asarray(self.model.times), y=array[i])

        return funcs

    def predict_survival_function(self, X):
        survivals = self.model.predict_survival(X)
        return self._array_to_step_function(survivals)

    def predict_cumulative_hazard_function(self, X):
        hazards = self.model.predict_cumulative_hazard(X)
        return self._array_to_step_function(hazards)

    def predict(self, X):
        risks = self.model.predict_risk(X)
        return risks

    def save(self, path):

        with open(os.path.join(path, "config.json"), "w", encoding="utf8") as f:
            json.dump(self.init_kwargs, f)
        with open(os.path.join(path, "train.json"), "w", encoding="utf8") as f:
            json.dump(self.fit_kwargs, f)

        self.model.save(os.path.join(path, 'model'))

    def load(self, path):
        if self.model is None:
            with open(os.path.join(path, "config.json"), "r", encoding="utf8") as f:
                self.init_kwargs = json.load(f)
            with open(os.path.join(path, "train.json"), "r", encoding="utf8") as f:
                self.fit_kwargs = json.load(f)

            self.model = self.base_model(**self.init_kwargs)

        self.model.load(os.path.join(path, 'model'))

class ConditionalSurvivalForestModel(PySurviavalWrapper):
    base_model = PySurvConditionalSurvivalForestModel

class NeuralMultiTaskModel(PySurviavalWrapper):
    base_model = PySurvNeuralMultiTaskModel

class NonLinearCoxPHModel(PySurviavalWrapper):
    base_model = PySurvNonLinearCoxPHModel

CSF_PARAMS = encode_params({
    "init": {
        "num_trees": [10, 50, 100, 200]
    },
    "fit": {
        "max_features": [
            "sqrt", "log2", "all",
        ] + [i/10 for i in range(10)],
        "min_node_size": [5, 10, 20, 50],
        "alpha": [0.01, 0.05, 0.1, 0.2],
        "minprop": [0.05, 0.1, 0.15, 0.2]
    }
})

NMTL_PARAMS = encode_params({
    "init": {
        "bins": [50, 100],
        "num_units": [64, 128, 256, 512, 1024],  # Number of hidden units per layer
        "activation": [
            "Atan", "BentIdentity", "BipolarSigmoid", "ELU", "Gaussian", "Hardtanh",
            "InverseSqrt", "LeakyReLU", "LeCunTanh", "LogLog", "LogSigmoid", "ReLU",
            "SELU", "Sigmoid", "Sinc", "Softmax", "Softplus", "Softsign", "Swish", "Tanh"
        ],
        "layers": [1, 2, 3],
    },
    "fit": {
        "init_method": [
            "glorot_uniform", "he_uniform", "uniform",
            "glorot_normal", "he_normal", "normal",
            "ones", "zeros", "orthogonal"
        ],
        "optimizer": [
            "adadelta", "adagrad", "adam",
            "adamax", "rmsprop", "sparseadam", "sgd"
        ],
        "lr": [i * (10 ** (-p)) for p in range(6, 2, -1) for i in range(1, 11)],
        "num_epochs": [100, 200, 500],
        "dropout": [i/10 for i in range(10)],
        "l2_reg": [i * (10 ** (-p)) for p in range(7, 4, -1) for i in range(1, 11)],
        "l2_smooth": [i * (10 ** (-p)) for p in range(7, 4, -1) for i in range(1, 11)],
        "bn_and_dropout": [True, False],
        "batch_normalization": [True, False]
    }
})

COX_PARAMS = encode_params({
    "init": {
        "num_units": [64, 128, 256, 512, 1024],  # Number of hidden units per layer
        "activation": [
            "Atan", "BentIdentity", "BipolarSigmoid", "ELU", "Gaussian", "Hardtanh",
            "InverseSqrt", "LeakyReLU", "LeCunTanh", "LogLog", "LogSigmoid", "ReLU",
            "SELU", "Sigmoid", "Sinc", "Softmax", "Softplus", "Softsign", "Swish", "Tanh"
        ],
        "layers": [1, 2, 3]
    },
    "fit": {
        "init_method": [
            "glorot_uniform", "he_uniform", "uniform",
            "glorot_normal", "he_normal", "normal",
            "ones", "zeros", "orthogonal"
        ],
        "optimizer": [
            "adadelta", "adagrad", "adam",
            "adamax", "rmsprop", "sparseadam", "sgd"
        ],
        "lr": [1e-4, 1e-3, 1e-2, 1e-1],
        "num_epochs": [100, 200, 500],
        "dropout": [i/10 for i in range(10)],
        "l2_reg": [i * (10 ** (-p)) for p in range(7, 4, -1) for i in range(1, 11)],
        "bn_and_dropout": [True, False],
        "batch_normalization": [True, False]
    }
})

In [ ]:
# %%capture

from survivors.experiments import grid as exp
import os

l_metrics = ["CI", "IBS", "IAUC"]
experim = exp.Experiments(folds=5, mode="CV+SAMPLE", except_stop=False)

experim.set_metrics(l_metrics)

experim.add_metric_best("CI")
experim.add_metric_best("IBS")

experim.add_method(ConditionalSurvivalForestModel, CSF_PARAMS)
experim.add_method(NeuralMultiTaskModel, NMTL_PARAMS)
experim.add_method(NonLinearCoxPHModel, COX_PARAMS)

experim.run_effective(X, y, verbose=0, stratify_best=[], save_model=True, dir_path=model_path)
# experim.run_effective(X, y, verbose=0, stratify_best=[], save_model=True)

experim.sample_table.to_csv(os.path.join(model_path, "results.csv"))

<class '__main__.ConditionalSurvivalForestModel'> {'init|num_trees': [10, 50, 100, 200], 'fit|max_features': ['sqrt', 'log2', 'all', 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 'fit|min_node_size': [5, 10, 20, 50], 'fit|alpha': [0.01, 0.05, 0.1, 0.2], 'fit|minprop': [0.05, 0.1, 0.15, 0.2]}
Method: ConditionalSurvivalForestModel, Param: {'fit|alpha': 0.01, 'fit|max_features': 0.0, 'fit|min_node_size': 5, 'fit|minprop': 0.05, 'init|num_trees': 10} finished with except 'max features is a positive value'
Method: ConditionalSurvivalForestModel, Param: {'fit|alpha': 0.01, 'fit|max_features': 0.0, 'fit|min_node_size': 5, 'fit|minprop': 0.05, 'init|num_trees': 50} finished with except 'max features is a positive value'
Method: ConditionalSurvivalForestModel, Param: {'fit|alpha': 0.01, 'fit|max_features': 0.0, 'fit|min_node_size': 5, 'fit|minprop': 0.05, 'init|num_trees': 100} finished with except 'max features is a positive value'
Method: ConditionalSurvivalForestModel, Param: {'fit|alp

<class '__main__.NeuralMultiTaskModel'> {'init|bins': [50, 100], 'init|num_units': [64, 128, 256, 512, 1024], 'init|activation': ['Atan', 'BentIdentity', 'BipolarSigmoid', 'ELU', 'Gaussian', 'Hardtanh', 'InverseSqrt', 'LeakyReLU', 'LeCunTanh', 'LogLog', 'LogSigmoid', 'ReLU', 'SELU', 'Sigmoid', 'Sinc', 'Softmax', 'Softplus', 'Softsign', 'Swish', 'Tanh'], 'init|layers': [1, 2, 3], 'fit|init_method': ['glorot_uniform', 'he_uniform', 'uniform', 'glorot_normal', 'he_normal', 'normal', 'ones', 'zeros', 'orthogonal'], 'fit|optimizer': ['adadelta', 'adagrad', 'adam', 'adamax', 'rmsprop', 'sparseadam', 'sgd'], 'fit|lr': [1e-06, 2e-06, 3e-06, 4e-06, 4.9999999999999996e-06, 6e-06, 7e-06, 8e-06, 9e-06, 9.999999999999999e-06, 1e-05, 2e-05, 3.0000000000000004e-05, 4e-05, 5e-05, 6.000000000000001e-05, 7.000000000000001e-05, 8e-05, 9e-05, 0.0001, 0.0001, 0.0002, 0.00030000000000000003, 0.0004, 0.0005, 0.0006000000000000001, 0.0007, 0.0008, 0.0009000000000000001, 0.001, 0.001, 0.002, 0.003, 0.004, 0.00

Method: NeuralMultiTaskModel, Param: {'fit|batch_normalization': True, 'fit|bn_and_dropout': True, 'fit|dropout': 0.0, 'fit|init_method': 'glorot_uniform', 'fit|l2_reg': 1e-07, 'fit|l2_smooth': 1e-07, 'fit|lr': 1e-06, 'fit|num_epochs': 100, 'fit|optimizer': 'adadelta', 'init|activation': 'BentIdentity', 'init|bins': 50, 'init|layers': 3, 'init|num_units': 512} finished with except 'The gradient exploded... You should reduce the learningrate (lr) of your optimizer'


Method: NeuralMultiTaskModel, Param: {'fit|batch_normalization': True, 'fit|bn_and_dropout': True, 'fit|dropout': 0.0, 'fit|init_method': 'glorot_uniform', 'fit|l2_reg': 1e-07, 'fit|l2_smooth': 1e-07, 'fit|lr': 1e-06, 'fit|num_epochs': 100, 'fit|optimizer': 'adadelta', 'init|activation': 'BentIdentity', 'init|bins': 100, 'init|layers': 1, 'init|num_units': 128} finished with except 'The gradient exploded... You should reduce the learningrate (lr) of your optimizer'


Method: NeuralMultiTaskModel, Param: {'fit|batch_normalization': True, 'fit|bn_and_dropout': True, 'fit|dropout': 0.0, 'fit|init_method': 'glorot_uniform', 'fit|l2_reg': 1e-07, 'fit|l2_smooth': 1e-07, 'fit|lr': 1e-06, 'fit|num_epochs': 100, 'fit|optimizer': 'adadelta', 'init|activation': 'BentIdentity', 'init|bins': 100, 'init|layers': 3, 'init|num_units': 256} finished with except 'The gradient exploded... You should reduce the learningrate (lr) of your optimizer'


...

Method: NeuralMultiTaskModel, Param: {'fit|batch_normalization': True, 'fit|bn_and_dropout': True, 'fit|dropout': 0.0, 'fit|init_method': 'glorot_uniform', 'fit|l2_reg': 1e-07, 'fit|l2_smooth': 1e-07, 'fit|lr': 1e-06, 'fit|num_epochs': 100, 'fit|optimizer': 'adamax', 'init|activation': 'ELU', 'init|bins': 50, 'init|layers': 2, 'init|num_units': 512} finished with except 'elu(): argument 'alpha' (position 2) must be Number, not NoneType'
Method: NeuralMultiTaskModel, Param: {'fit|batch_normalization': True, 'fit|bn_and_dropout': True, 'fit|dropout': 0.0, 'fit|init_method': 'glorot_uniform', 'fit|l2_reg': 1e-07, 'fit|l2_smooth': 1e-07, 'fit|lr': 1e-06, 'fit|num_epochs': 100, 'fit|optimizer': 'adamax', 'init|activation': 'ELU', 'init|bins': 50, 'init|layers': 2, 'init|num_units': 1024} finished with except 'elu(): argument 'alpha' (position 2) must be Number, not NoneType'
Method: NeuralMultiTaskModel, Param: {'fit|batch_normalization': True, 'fit|bn_and_dropout': True, 'fit|dropout': 0.0

Method: NeuralMultiTaskModel, Param: {'fit|batch_normalization': True, 'fit|bn_and_dropout': True, 'fit|dropout': 0.0, 'fit|init_method': 'glorot_uniform', 'fit|l2_reg': 1e-07, 'fit|l2_smooth': 1e-07, 'fit|lr': 1e-06, 'fit|num_epochs': 100, 'fit|optimizer': 'adamax', 'init|activation': 'ELU', 'init|bins': 50, 'init|layers': 3, 'init|num_units': 512} finished with except 'elu(): argument 'alpha' (position 2) must be Number, not NoneType'
Method: NeuralMultiTaskModel, Param: {'fit|batch_normalization': True, 'fit|bn_and_dropout': True, 'fit|dropout': 0.0, 'fit|init_method': 'glorot_uniform', 'fit|l2_reg': 1e-07, 'fit|l2_smooth': 1e-07, 'fit|lr': 1e-06, 'fit|num_epochs': 100, 'fit|optimizer': 'adamax', 'init|activation': 'ELU', 'init|bins': 50, 'init|layers': 3, 'init|num_units': 1024} finished with except 'elu(): argument 'alpha' (position 2) must be Number, not NoneType'
Method: NeuralMultiTaskModel, Param: {'fit|batch_normalization': True, 'fit|bn_and_dropout': True, 'fit|dropout': 0.0

Method: NeuralMultiTaskModel, Param: {'fit|batch_normalization': True, 'fit|bn_and_dropout': True, 'fit|dropout': 0.0, 'fit|init_method': 'glorot_uniform', 'fit|l2_reg': 1e-07, 'fit|l2_smooth': 1e-07, 'fit|lr': 1e-06, 'fit|num_epochs': 100, 'fit|optimizer': 'adamax', 'init|activation': 'ELU', 'init|bins': 100, 'init|layers': 1, 'init|num_units': 512} finished with except 'elu(): argument 'alpha' (position 2) must be Number, not NoneType'
Method: NeuralMultiTaskModel, Param: {'fit|batch_normalization': True, 'fit|bn_and_dropout': True, 'fit|dropout': 0.0, 'fit|init_method': 'glorot_uniform', 'fit|l2_reg': 1e-07, 'fit|l2_smooth': 1e-07, 'fit|lr': 1e-06, 'fit|num_epochs': 100, 'fit|optimizer': 'adamax', 'init|activation': 'ELU', 'init|bins': 100, 'init|layers': 1, 'init|num_units': 1024} finished with except 'elu(): argument 'alpha' (position 2) must be Number, not NoneType'
Method: NeuralMultiTaskModel, Param: {'fit|batch_normalization': True, 'fit|bn_and_dropout': True, 'fit|dropout': 0

Method: NeuralMultiTaskModel, Param: {'fit|batch_normalization': True, 'fit|bn_and_dropout': True, 'fit|dropout': 0.0, 'fit|init_method': 'glorot_uniform', 'fit|l2_reg': 1e-07, 'fit|l2_smooth': 1e-07, 'fit|lr': 1e-06, 'fit|num_epochs': 100, 'fit|optimizer': 'adamax', 'init|activation': 'ELU', 'init|bins': 100, 'init|layers': 2, 'init|num_units': 512} finished with except 'elu(): argument 'alpha' (position 2) must be Number, not NoneType'
Method: NeuralMultiTaskModel, Param: {'fit|batch_normalization': True, 'fit|bn_and_dropout': True, 'fit|dropout': 0.0, 'fit|init_method': 'glorot_uniform', 'fit|l2_reg': 1e-07, 'fit|l2_smooth': 1e-07, 'fit|lr': 1e-06, 'fit|num_epochs': 100, 'fit|optimizer': 'adamax', 'init|activation': 'ELU', 'init|bins': 100, 'init|layers': 2, 'init|num_units': 1024} finished with except 'elu(): argument 'alpha' (position 2) must be Number, not NoneType'
Method: NeuralMultiTaskModel, Param: {'fit|batch_normalization': True, 'fit|bn_and_dropout': True, 'fit|dropout': 0

Method: NeuralMultiTaskModel, Param: {'fit|batch_normalization': True, 'fit|bn_and_dropout': True, 'fit|dropout': 0.0, 'fit|init_method': 'glorot_uniform', 'fit|l2_reg': 1e-07, 'fit|l2_smooth': 1e-07, 'fit|lr': 1e-06, 'fit|num_epochs': 100, 'fit|optimizer': 'adamax', 'init|activation': 'ELU', 'init|bins': 100, 'init|layers': 3, 'init|num_units': 512} finished with except 'elu(): argument 'alpha' (position 2) must be Number, not NoneType'
Method: NeuralMultiTaskModel, Param: {'fit|batch_normalization': True, 'fit|bn_and_dropout': True, 'fit|dropout': 0.0, 'fit|init_method': 'glorot_uniform', 'fit|l2_reg': 1e-07, 'fit|l2_smooth': 1e-07, 'fit|lr': 1e-06, 'fit|num_epochs': 100, 'fit|optimizer': 'adamax', 'init|activation': 'ELU', 'init|bins': 100, 'init|layers': 3, 'init|num_units': 1024} finished with except 'elu(): argument 'alpha' (position 2) must be Number, not NoneType'


In [ ]:
import pandas as pd

for idx, row in experim.sample_table[['METHOD','FOLD_IDX'] + experim.metric_best_p].iterrows():
    method_name = f" {row['METHOD']} "

    # flattern df
    sub_table = row[['FOLD_IDX'] + experim.metric_best_p].to_dict()
    sub_table = pd.DataFrame(sub_table)

    ascending_list = [c != "max" for c in experim.way_best_p]

    def compute_sort_value(row, metrics, ascending_list):
        values = []

        for metric, is_ascending in zip(metrics, ascending_list):
            values.append(1 - row[metric] if is_ascending else row[metric])

        return sum(values) / len(values)

    sub_table['SORT_VALUE'] = sub_table.apply(lambda row: compute_sort_value(row, experim.metric_best_p, ascending_list), axis = 1)
    sub_table_sorted = sub_table.sort_values(by='SORT_VALUE', ascending=False)

    best_row = sub_table_sorted.iloc[0]
    best_fold = int(best_row['FOLD_IDX'])

    best_model_path = os.path.join(model_path, method_name, f"{best_fold}")

    print(f"'{method_name:=^100}'\n")
    print(best_model_path)
    print()
    print(best_row)
    print()
    print(sub_table_sorted.to_markdown())
    print()
    print()

In [ ]:
# Create training and testing sets
X = df[features]
T = df[cnt.TIME_NAME].values
E = df[cnt.CENS_NAME].values



print(X_test.shape)
print(T_test.shape)
print(E_test.shape)

In [ ]:
# Initialize and fit the Conditional Survival Forest model
csf = PySurvConditionalSurvivalForestModel(num_trees=200)

In [ ]:
csf.fit(
    X_train, T_train, E_train,
    max_features="sqrt", max_depth=5, min_node_size=20,
    alpha=0.05, minprop=0.1,
    save_memory = True, seed = 42,
    verbose = True
)

In [ ]:
np.asarray(csf.times)

In [ ]:
csf.predict_risk(X_test)

In [ ]:
survs = csf.predict_survival(X_test)

temp = [ClosestBin(surv, csf.times) for surv in survs]

In [ ]:
survs.shape

In [ ]:
temp[0](T_test[3])

In [ ]:
hazard = csf.predict_hazard(X_test)

In [ ]:
from pysurvival.models.multi_task import NeuralMultiTaskModel

n_mtlr = NeuralMultiTaskModel(
    bins=100,
    structure=[
        {'activation': 'ReLU', 'num_units': 500},
        {'activation': 'ReLU', 'num_units': 500},
        {'activation': 'ReLU', 'num_units': 500}
    ],
)

n_mtlr.fit(
    X_train, T_train, E_train,
    lr = 0.0001,
    num_epochs = 500,
    l2_reg = 0.000100,
    l2_smooth = 0.000100,
    optimizer = 'rprop',
    init_method = 'glorot_uniform'
)

# Model evaluation

In [ ]:
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score, compare_to_actual

def evaluate(model, X, T, E, model_name="", plot=True):
    errors = compare_to_actual(model, X, T, E, is_at_risk=False, plot=plot)
    c_index = concordance_index(model, X, T, E)
    ibs = integrated_brier_score(model, X, T, E, plot=plot)

    metrics = {
        'C-index': c_index,
        'IBS': ibs,
        'RMSE': errors['root_mean_squared_error'],
        'MADE': errors['median_absolute_error'],
        'MAE': errors['mean_absolute_error']
    }

    eval_df = pd.DataFrame(data=metrics, index=[model_name])

    return eval_df, metrics

eval_df, metrics = evaluate(csf, X_test, T_test, E_test, plot=True)
eval_dfAvg

In [ ]:
eval_df, metrics = evaluate(n_mtlr, X_test, T_test, E_test, plot=True)
eval_df

In [ ]:
# Plotting survival curves for a sample data point
fig, ax = plt.subplots(figsize=(8, 4))

# Randomly extract a data point that experienced an event
choices = np.argwhere((E_test == 1) & (T_test >= 1)).flatten()
k = np.random.choice(choices, 1)[0]

# Time of the event
t = T_test[k]

# Compute and plot the survival function
survival = n_mtlr.predict_survival(X_test.values[k, :]).flatten()

# Display the functions
plt.plot(n_mtlr.times, survival, color='blue', label='Predicted', lw=4, ls='-.')

# Highlight the true time of the event in red
plt.axvline(x=t, color='red', ls='--', label='True Time')
ax.annotate('T={:.1f}'.format(t), xy=(t, 0.5), xytext=(t, 0.5), fontsize=12, color='red')

# Show everything
title = "Predicted Survival Function for a Data Point"
plt.legend(fontsize=12)
plt.title(title, fontsize=15)
plt.ylim(0, 1.05)
plt.xlabel('Time')
plt.ylabel('Survival Probability')
plt.show()
